# Data Driven Science and Engineering
## Chapter 2 - Fourier Transformations
To see explinations of Fourier Transformations, please see the markdown file within this folder

----
importing libraries

In [1]:
import numpy as np
import pandas as pd

----
Example Fourier Series for a continuous Hat Function


$\Biggl\{\begin{align*} 
0 \;for\; x \in [-\pi,-\pi/2 ) \\ 
1+2x/\pi \;for\; x \in [-\pi/2,0) \\
1-sx/\pi \;for\; x\in [0,\pi/2)\\
0 \;for\; x \in [\pi/2,\pi)
\end{align*}$

In [6]:
# deinfe domain
dx = 0.001
L = np.pi
x = np.arange(-1,1,dx) * L
n = x.size

# deginign hat function
